In [1]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [2]:
import copy
import datetime

# Loading the 2 dataset

In [3]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [4]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [5]:
df=pd.concat([df1,df2],axis=0)

In [6]:
df.head()

,cat,sex,rang,nom,an,lieu,temps,retard,pace,link
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,http://services.datasport.com/1999/lauf/biel/A...
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,http://services.datasport.com/1999/lauf/biel/A...
2,Klasse8,False,3,"Abduli Ajete, Biel/Bienne",84,Schule Sahligut Hinterman,"10.50,4","1.54,6",False,http://services.datasport.com/1999/lauf/biel/A...
3,Klasse8,False,9,"Abou Al Nassr Chaled, Biel/Bienne",84,Schule Sahligut Hinterman,"14.09,9","5.14,1",False,http://services.datasport.com/1999/lauf/biel/A...
4,H/M35,M,1,Abplanalp Michael,64,Bern,"1:16.53,8",---,False,http://services.datasport.com/1999/lauf/biel/A...


In [7]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [8]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [9]:
links2runs=links2runs.set_index(links2runs.URL)

In [10]:
links2runs.head(3)

,Date,Name,Place,URL
URL,,,,
http://services.datasport.com/1999/zkb/maennedorf,sam. 27.03.1999,Männedörfler Waldlauf,Männedorf,http://services.datasport.com/1999/zkb/maennedorf
http://services.datasport.com/1999/lauf/kerzers,sam. 20.03.1999,Kerzerslauf,Kerzers,http://services.datasport.com/1999/lauf/kerzers
http://services.datasport.com/1999/lauf/luzern,sam. 24.04.1999,Luzerner Stadtlauf,Luzern,http://services.datasport.com/1999/lauf/luzern


In [11]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

,Date,Name,Place,URL
URL,,,,
http://services.datasport.com/2000/lauf/tdcn,mer. 26.04.2000,BCN Tour St. Aubin,Chézard-St-Martin,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 19.04.2000,BCN Tour Le Landeron,Colombier NE,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 24.05.2000,BCN Tour Neuchâtel,Neuchâtel,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 17.05.2000,BCN Tour La Sagne,Couvet,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 10.05.2000,BCN Tour Dombresson,La Chaux-de-Fonds,http://services.datasport.com/2000/lauf/tdcn
http://services.datasport.com/2000/lauf/tdcn,mer. 03.05.2000,BCN Tour Noiraigue,Les Ponts-de-Martel,http://services.datasport.com/2000/lauf/tdcn


In [12]:
links2runs.index.value_counts()

http://services.datasport.com/2000/lauf/tdcn                      6
http://services.datasport.com/2002/lauf/tdpv/etape3               2
http://services.datasport.com/2011/lauf/tdpv/etape4               2
http://services.datasport.com/2007/lauf/zueriwalkt                2
http://services.datasport.com/2009/lauf/defi                      2
http://services.datasport.com/2009/lauf/tdpv/etape4               2
http://services.datasport.com/2003/lauf/tdpv/etape3               2
http://services.datasport.com/2007/diverse/tzampata               2
http://services.datasport.com/2010/lauf/tdpv/etape4               2
http://services.datasport.com/2006/lauf/morat                     2
http://services.datasport.com/2011/lauf/willisau                  1
http://services.datasport.com/2014/lauf/montreux                  1
http://services.datasport.com/2002/lauf/tdcn/etape2               1
http://services.datasport.com/2014/lauf/tdpv/etape6               1
http://services.datasport.com/2012/lauf/sempach 

### Process link

In [13]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [14]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,link
index,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,http://services.datasport.com/1999/lauf/biel
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,http://services.datasport.com/1999/lauf/biel


### Groupby link

In [15]:
links=list(df.link.value_counts().index)

In [16]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
    try:
        row_links2runs.shape[1]
        if row_links2runs.shape[0]>1:
            row_links2runs=row_links2runs.iloc[0]
    except:
        pass
#     print(row_links2runs.shape)
    try:
        x['Date']=row_links2runs.Date
    except:
        print(row_links2runs)
        print(x)
        raise()
    x['Name']=row_links2runs.Name
    return x

In [17]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,link
index,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,http://services.datasport.com/1999/lauf/biel
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,http://services.datasport.com/1999/lauf/biel


In [18]:
grouped_link=df.groupby(df.link)

In [19]:
df=grouped_link.apply(add_date_name_to_group)

In [20]:
del df['link']

In [21]:
df.head(2)

,cat,sex,rang,nom,an,lieu,temps,retard,pace,Date,Name
index,,,,,,,,,,,
0,M/M30,M,16,Abderhalden Seth,69,Luzern,"3:33.32,9","37.38,2",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne"
1,Klasse8,False,8,"Abdirizak Ayanle, Biel/Bienne",84,Schule Sahligut Hinterman,"14.02,3","5.06,5",False,ven. 11.06.1999,"Bieler Lauftage, Biel/Bienne"


# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [22]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [23]:
class Time:
    def __init__(self,time,mseconds=True):
        self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]
        hours+=int(minutes/60)
        minutes=minutes%60
        if not check_only:
            return (hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [24]:
dict_problems={}

In [ ]:
re.split("[:.]+",'14.46')

In [ ]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [ ]:
df.temps=df.temps.apply(apply_time)

In [ ]:
df.retard=df.retard.apply(apply_time)

In [ ]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [ ]:
print(Time('95.12,2'))

In [ ]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

In [ ]:
b.get_seconds()

In [ ]:
a.get_seconds()

In [ ]:
a/b

In [ ]:
df['Distance']=df.temps/df.pace

In [ ]:
df.head(2)

In [ ]:
df_old=df.copy()

In [ ]:
df=df[df.Distance>0]

In [ ]:
df_old.shape

In [ ]:
df=df_old[df_old.Distance>0]

In [ ]:
df.head(3)

# Parse Date

In [ ]:
df['Year']=df.Date.apply(lambda x: x.split('.')[3])

In [ ]:
df['Month']=df.Date.apply(lambda x: x.split('.')[2])

In [ ]:
df.head(2)

# Sex Male-Female

### Get names database

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
gender_names=pd.read_csv('gender_names.csv')

In [ ]:
del gender_names['Unnamed: 0']

In [ ]:
gender_names=gender_names.set_index(gender_names.name)

In [ ]:
del gender_names['name']

In [ ]:
gender_names.head(2)

### Category from sex database

In [ ]:
category_list=list(df.cat.value_counts().index)

In [ ]:
len(category_list)

In [ ]:
category_dict_sex={}

In [ ]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [ ]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

In [ ]:
len(cat_to_check)

In [ ]:
len(category_dict_sex)

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [ ]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

### Improvements

In [ ]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [ ]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

In [ ]:
df[df.cat=='Sch-D-M']

### Final classification - accurate

In [ ]:
del df['sex']

In [ ]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [ ]:
grouped=df.groupby(df.cat)

In [ ]:
df=grouped.apply(add_sex)

In [ ]:
df.sex.value_counts()

In [ ]:
df_sex=df[df.sex!=False]

In [ ]:
df_sex.head(2)

In [ ]:
df.shape

In [ ]:
df_sex.shape

In [ ]:
min(df.Year)

In [ ]:
min(df_sex.Year)

In [ ]:
df=df.rename(columns={'Year':'RaceYear','Month':'RaceMonth'})

In [ ]:
df.head(2)

In [ ]:
df.sex.value_counts()

# Parse year (roughly eliminating clearly bad entries)

In [ ]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [ ]:
df.an=df.an.apply(lambda x: int(x))

In [ ]:
df=df[df.an<2017]

In [ ]:
df=df[(df.an>1870) | (df.an<100)]

# Columns names

In [ ]:
df.head(1)

In [ ]:
df.columns.values

In [ ]:
df=df[['Name','RaceYear','RaceMonth','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [ ]:
df.head()

In [ ]:
df.columns=['Race','RaceYear','RaceMonth','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [ ]:
df=df.reset_index()

In [ ]:
del df['index']

In [ ]:
df.head(2)

In [ ]:
df.shape

# Distance

In [ ]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [ ]:
df['round']=df.Distance.round()

In [ ]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [ ]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [ ]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [ ]:
df.Distance.value_counts()

In [ ]:
del df['round']

In [ ]:
df.head(5)

# Adding weather informations

In [ ]:
df=df[df.Sex!=False]

In [ ]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [ ]:
weather_db.head(2)

In [ ]:
weather_db['RaceYear']=weather_db.Date.apply(lambda x: x.split('.')[3])

In [ ]:
gr=df.groupby([df.RaceYear,df.Race])

In [ ]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].RaceYear
    weather=weather_db[(weather_db.Name==race) & (weather_db.RaceYear==date)].iloc[0]
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [ ]:
df=gr.apply(merge_weather_info)

In [ ]:
df.shape

In [ ]:
df.head(2)

# Parse times and select only meaningful features

In [ ]:
full_df=df.copy()

In [ ]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:Time(x))
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:Time(x))
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%H:%M.%S,%f')-base)

In [ ]:
max(full_df.Distance)

In [ ]:
full_df=full_df[full_df.Distance<45]

In [ ]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%H:%M.%S,%f')-base)

In [ ]:
del full_df['Delay']

In [ ]:
full_df.head()

# Better distance processing

In [ ]:
df.Race.value_counts()

In [ ]:
b=full_df2[full_df2.Race=='20km de Lausanne']

In [ ]:
b.Distance.value_counts().index

In [ ]:
full_df2=full_df.copy()

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: x if x<4 or (abs(1-x/round(x)))>0.0055 else round(x))

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 21.0975 if x>4 and (abs(1-x/21.0975))<0.0055 else x)

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 42.195 if x>4 and (abs(1-x/42.195))<0.0055 else x)

In [ ]:
full_df2.groupby(full_df2.RaceYear).count()

In [ ]:
df=full_df2.copy()

# Year - trying to get better data

In [ ]:
years_values=df.Year.value_counts().index.values
years_values[years_values<120]

In [ ]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

In [ ]:
df[df.Year>1900].shape

In [ ]:
df[df.Year<1900].shape

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
reduced_df[reduced_df.Year==25]

In [ ]:
reduced_df.groupby(reduced_df.Year).count()

If in the name a comma is present, it is possible that the year is misunderstood.

In [ ]:
names=reduced_df.Name
list_names=[]
for name in names:
    if ',' in name:
        list_names.append(name)

In [ ]:
df['CommaInName']=df.Name.str.contains(',')

In [ ]:
df['Less120']=df.Year<120

In [ ]:
df['FalseYear']=df.CommaInName & df.Less120

In [ ]:
def false_year(x):
    if x.iloc[0].FalseYear:
        x['Year']=float('nan')
    return x

In [ ]:
df=df.groupby(df.FalseYear).apply(false_year)

In [ ]:
del df['CommaInName']
del df['Less120']
del df['FalseYear']

In [ ]:
df.head(4)

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
list(reduced_df.Year.value_counts().index)

In [ ]:
df.Year=df.Year.apply(lambda x: float('nan') if x<10 else x)

# Output
First output

In [ ]:
df_output=df.copy()

In [ ]:
df_output.shape

In [ ]:
df_output.head()

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

In [ ]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

In [ ]:
df_output

In [ ]:
df_output.len_name.value_counts()

In [ ]:
df_output[df_output.len_name==6]

In [ ]:
df_output.groupby(df_output.RaceYear).count()

In [ ]:
df.groupby(df.RaceYear).count()

# Solve problem 2 runners with same name in the same race

In [ ]:
df_output=pd.read_pickle('../datasets/df_year_fixed.pickle')

In [ ]:
df_red=df_output[df_output.Race!='Course de l\'Escalade, Genève']

In [ ]:
df_red.shape

In [ ]:
grouped=df_red.groupby([df_red.RaceID,df_red.Name,df_output.Year])

In [ ]:
cont=0
for i,a in grouped:
    cont+=1
    if cont%100000==0:
        print(cont)
    if cont>1000000:
        if a.shape[0]>1:
            print(a)
    
    

# Sex verification

In [ ]:
lau=df_output[(df_output.RaceID=='http://services.datasport.com/2007/lauf/Greifenseelauf')&(df_output.Distance>6)]

In [ ]:
lau.Distance.value_counts()

In [ ]:
lau.Sex.value_counts()

In [ ]:
df_output.RaceID.value_counts()

In [ ]:
df_output

# Output to file

In [ ]:
df_output.to_pickle('../datasets/df_year_fixed.pickle')

In [ ]:
df_output